In [10]:
from influxdb_client import InfluxDBClient

url = "http://localhost:8086"

# hot
token = "gCGWIFyp0f0WxPnz3w_k7N993EPqvf-4qit7H8XMs5eoPJx4LdUHNnrc371Vvd6hYAdrBcEm_k89HlIAwUk3LA=="
org = "user"

# nob
token = "JYn7HQ4YRosmEdIAKonWE5KeMNhu28e-p1erovFlDgQPkqbkinUZ_ty-uDbL2eNvuHs_RFE142p5lSi80qTDRg=="
org = "tclab"

bucket = "power"

import os 
import json

def read_(fname):
    rc = ""
    with open(fname) as f:
        rc = f.readlines()
    return "".join(rc)

def write_(tables, fname):
    
    output = json.loads(tables.to_json())

    ls = "\n"
    sep = "|"

    with open(fname, "w") as f:
   
        columns = list(output[0].keys())
        _ = [columns.remove(c) for c in ['result']]
        rc = sep.join(c for c in columns)
        f.write(f"{rc}{ls}")

        for row in output:
            rc = sep.join(str(row[c]) for c in columns)
            f.write(f"{rc}{ls}")

    print(f"wrote {len(output)} line(s) of {fname}")


In [11]:
%%time 

fpath = os.path.join(os.getcwd(),"queries")
fnames = {fname: os.path.join(fpath,fname) for fname in os.listdir(fpath) if fname.endswith(".flux")}
fqueries = {fname: read_(fnames[fname]) for fname in fnames.keys()}

CPU times: total: 0 ns
Wall time: 2.02 ms


In [12]:
%%time

fname = "234b.flux"

with InfluxDBClient(url=url, token=token, org=org, debug=False, timeout=240000) as client:
    
    query_api = client.query_api()        
    query = fqueries[fname]

    tables = query_api.query(org=org, query=query)
    write_(tables, fname=os.path.join(fpath,"%s.txt"%(fname)))


wrote 4 line(s) of c:\Develop\vsc\tclab\idb\queries\234b.flux.txt
CPU times: total: 0 ns
Wall time: 87.6 ms
